# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [9]:
import sqlalchemy

In [10]:
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine
from scipy import stats
import seaborn as sns

In [11]:
POSTGRES_ADDRESS = 'lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com' 
POSTGRES_PORT = '5432' 
POSTGRES_USERNAME = 'lhl_student' 
POSTGRES_PASSWORD = 'lhl_student' 
POSTGRES_DBNAME='mid_term_project'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, 
                                                                                        password=POSTGRES_PASSWORD, 
                                                                                        ipaddress=POSTGRES_ADDRESS, 
                                                                                        port=POSTGRES_PORT, 
                                                                                        dbname=POSTGRES_DBNAME))

In [12]:
cnx = create_engine(postgres_str)

In [13]:
import pandas as pd
import numpy as np
from scipy import stats

In [14]:
df_flight=pd.read_sql_query('''SELECT * FROM flights ORDER BY RANDOM() LIMIT 10000;''', cnx)

In [15]:
df_flight.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-05-31,UA,UA,UA,774,UA,N459UA,774,11292,DEN,...,692.0,0.0,0.0,36.0,0.0,0.0,None,NaN,NaN,None
1,2019-06-10,UA,UA,UA,2165,UA,N12754,2165,10721,BOS,...,200.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2018-08-17,AA,AA,AA,1439,AA,N940AN,1439,14107,PHX,...,2133.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2018-09-15,WN,WN,WN,4279,WN,N7725A,4279,14869,SLC,...,590.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
4,2019-06-04,AA,AA,AA,2066,AA,N916US,2066,14100,PHL,...,2370.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [16]:
df_flight=df_flight[['fl_date', 'air_time','mkt_unique_carrier', 'branded_code_share', 'mkt_carrier','mkt_carrier_fl_num','op_unique_carrier', 'tail_num','op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name','dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time','dep_delay','crs_arr_time', 'dup', 'crs_elapsed_time','flights', 'distance','arr_delay','cancelled','diverted']]

In [17]:
df_flight.isnull().sum()

fl_date                 0
air_time              202
mkt_unique_carrier      0
branded_code_share      0
mkt_carrier             0
mkt_carrier_fl_num      0
op_unique_carrier       0
tail_num               35
op_carrier_fl_num       0
origin_airport_id       0
origin                  0
origin_city_name        0
dest_airport_id         0
dest                    0
dest_city_name          0
crs_dep_time            0
dep_delay             166
crs_arr_time            0
dup                     0
crs_elapsed_time        0
flights                 0
distance                0
arr_delay             201
cancelled               0
diverted                0
dtype: int64

In [18]:
df_flight=df_flight.dropna()

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [19]:
# Converting date and time columns
df_flight['fl_date'] = pd.to_datetime(df_flight['fl_date'], errors='coerce')
df_flight['month'] = df_flight['fl_date'].dt.month
df_flight['day_of_week'] = df_flight['fl_date'].dt.dayofweek
df_flight['day_of_month'] = df_flight['fl_date'].dt.day
df_flight['year'] = df_flight['fl_date'].dt.year

In [20]:
# Creating columns for departure and arrival hours
df_flight['crs_dep_time'] = df_flight['crs_dep_time'].apply(lambda x: '{0:0>4}'.format(x))
df_flight['crs_arr_time'] = df_flight['crs_arr_time'].apply(lambda x: '{0:0>4}'.format(x))
df_flight['dep_hour']=df_flight['crs_dep_time'].str[:2]
df_flight['arr_hour']=df_flight['crs_arr_time'].str[:2]

In [21]:
# Dividing the months into four seasons
df_flight['seasons']='spring'
df_flight.loc[df_flight['month'].between(6, 8), 'seasons']= 'summer'
df_flight.loc[df_flight['month'].between(9, 11), 'seasons']= 'autum'
df_flight.loc[df_flight['month'].between(12, 2), 'seasons']= 'winter'

In [22]:
df_flight.head()

,fl_date,air_time,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,arr_delay,cancelled,diverted,month,day_of_week,day_of_month,year,dep_hour,arr_hour,seasons
0,2019-05-31,85.0,UA,UA,UA,774,UA,N459UA,774,11292,...,36.0,0.0,0.0,5,4,31,2019,11,14,spring
1,2019-06-10,51.0,UA,UA,UA,2165,UA,N12754,2165,10721,...,-4.0,0.0,0.0,6,0,10,2019,08,09,summer
2,2018-08-17,261.0,AA,AA,AA,1439,AA,N940AN,1439,14107,...,-3.0,0.0,0.0,8,4,17,2018,21,05,summer
3,2018-09-15,92.0,WN,WN,WN,4279,WN,N7725A,4279,14869,...,-9.0,0.0,0.0,9,5,15,2018,08,09,autum
4,2019-06-04,298.0,AA,AA,AA,2066,AA,N916US,2066,14100,...,-38.0,0.0,0.0,6,1,4,2019,18,21,summer


In [23]:
# Dividing the depature and arrival hours into day sections
df_flight['dep_hour']=df_flight['dep_hour'].astype(int)
df_flight['dep_time_during_day']='night'
df_flight.loc[df_flight['dep_hour'].between(12,18), 'dep_time_during_day']= 'afternoon'
df_flight.loc[df_flight['dep_hour'].between(19,23), 'dep_time_during_day']= 'evening'
df_flight.loc[df_flight['dep_hour'].between(4,11), 'dep_time_during_day']= 'morning'

df_flight['arr_hour']=df_flight['arr_hour'].astype(int)
df_flight['arr_time_during_day']='night'
df_flight.loc[df_flight['arr_hour'].between(12,18), 'arr_time_during_day']= 'afternoon'
df_flight.loc[df_flight['arr_hour'].between(19,23), 'arr_time_during_day']= 'evening'
df_flight.loc[df_flight['arr_hour'].between(4,11), 'arr_time_during_day']= 'morning'

In [24]:
df_flight = df_flight.drop('fl_date', axis = 1)

In [25]:
df_flight['dep_time_during_day'].value_counts()

afternoon    4109
morning      4032
evening      1622
night          31
Name: dep_time_during_day, dtype: int64

In [26]:
#Converting to air_time and arr_delay to category
def flight_duration(x):
    if x <=180:
        return 'Short'
    elif x >180 and x<360:
        return 'Medium'
    elif x>360:
        return 'Long'

df_flight['flight_duration_type']=df_flight['air_time'].apply(lambda x: flight_duration(x))
df_flight['flight_duration_type'].value_counts()

Short     8522
Medium    1235
Long        32
Name: flight_duration_type, dtype: int64

In [27]:
# Separating State name from city
df_flight['origin_state']=df_flight['origin_city_name'].apply(lambda x: x.split(', ')[1])
df_flight['destination_state']=df_flight['dest_city_name'].apply(lambda x: x.split(', ')[1])

In [28]:
# Finding the mean of departure and arrival delays based on different carriers
df_flight['dep_delay_mean_by_carrier']=df_flight.groupby(['op_unique_carrier'])['dep_delay'].transform(np.mean)
df_flight['arr_delay_mean_by_carrier']=df_flight.groupby(['op_unique_carrier'])['arr_delay'].transform(np.mean)

In [29]:
#encode arrival delay
df_flight['arr_delay_cat'] = df_flight['arr_delay'].apply(lambda x: 1 if x > 0 else 0)
# ENCODE AIRPORTS AND TAILNUM
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df_flight['mkt_carrier'] = encoder.fit_transform(df_flight[['mkt_carrier']])
df_flight['mkt_unique_carrier'] = encoder.fit_transform(df_flight[['mkt_unique_carrier']])
df_flight['mkt_carrier_fl_num'] = encoder.fit_transform(df_flight[['mkt_carrier_fl_num']])
df_flight['op_unique_carrier'] = encoder.fit_transform(df_flight[['op_unique_carrier']])
df_flight['tail_num'] = encoder.fit_transform(df_flight[['tail_num']])
df_flight['op_carrier_fl_num'] = encoder.fit_transform(df_flight[['op_carrier_fl_num']])
df_flight['origin_airport_id'] = encoder.fit_transform(df_flight[['origin_airport_id']])
df_flight['dest_airport_id'] = encoder.fit_transform(df_flight[['dest_airport_id']])

In [30]:
# Finding the air traffics based on the airports
df_flight['origin_airport_taffic']=df_flight.groupby(['origin_airport_id'])['flights'].transform(np.sum)
df_flight['dest_airport_traffic']=df_flight.groupby(['dest_airport_id'])['flights'].transform(np.sum)

In [31]:
# Converting categorical data by getting dummies
dummy_durtypes=pd.get_dummies(df_flight[['flight_duration_type', 'dep_time_during_day', 'arr_time_during_day', 'seasons']])

df_flight=pd.concat([df_flight, dummy_durtypes], axis=1).drop(['flight_duration_type', 'dep_time_during_day', 'arr_time_during_day', 'seasons'], axis=1)

In [32]:
df_flight.isnull().sum()

air_time                         0
mkt_unique_carrier               0
branded_code_share               0
mkt_carrier                      0
mkt_carrier_fl_num               0
op_unique_carrier                0
tail_num                         0
op_carrier_fl_num                0
origin_airport_id                0
origin                           0
origin_city_name                 0
dest_airport_id                  0
dest                             0
dest_city_name                   0
crs_dep_time                     0
dep_delay                        0
crs_arr_time                     0
dup                              0
crs_elapsed_time                 0
flights                          0
distance                         0
arr_delay                        0
cancelled                        0
diverted                         0
month                            0
day_of_week                      0
day_of_month                     0
year                             0
dep_hour            

In [33]:
df_flight=df_flight.drop(['dep_delay','mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
                            'mkt_carrier_fl_num', 'op_unique_carrier','op_carrier_fl_num', 
                            'origin', 'origin_city_name','dup','dest','dest_city_name',
                            'origin_state', 'destination_state', 
                            'cancelled','diverted',
                           ], axis=1)

In [34]:
df_flight.head()

,air_time,tail_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay,...,dep_time_during_day_evening,dep_time_during_day_morning,dep_time_during_day_night,arr_time_during_day_afternoon,arr_time_during_day_evening,arr_time_during_day_morning,arr_time_during_day_night,seasons_autum,seasons_spring,seasons_summer
0,85.0,1634.0,77.0,56.0,1115,1403,108.0,1.0,692.0,36.0,...,0,1,0,1,0,0,0,0,1,0
1,51.0,199.0,39.0,99.0,0800,0918,78.0,1.0,200.0,-4.0,...,0,1,0,0,0,1,0,0,0,1
2,261.0,4188.0,231.0,99.0,2155,0533,278.0,1.0,2133.0,-3.0,...,1,0,0,0,0,1,0,0,0,1
3,92.0,2934.0,282.0,160.0,0805,0900,115.0,1.0,590.0,-9.0,...,0,1,0,0,0,1,0,1,0,0
4,298.0,3967.0,230.0,260.0,1833,2132,359.0,1.0,2370.0,-38.0,...,0,0,0,0,1,0,0,0,0,1


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [35]:
'''from sklearn.decomposition import PCA
import matplotlib.pyplot as plt'''

'from sklearn.decomposition import PCA\nimport matplotlib.pyplot as plt'

In [36]:
#YET TO COMPLETE

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [37]:
import pickle

* Target and Features

In [38]:
y = np.array(df_flight.arr_delay)

In [39]:
y_cat = np.array(df_flight.arr_delay_cat)

In [40]:
X_df = df_flight.drop(['arr_delay', 'arr_delay_cat'], axis = 1)

In [41]:
X = np.array(X_df)

* Train Test Split and Samples

In [42]:
import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.70,test_size=0.30, random_state=101, shuffle = True)

In [43]:
X_train_sample = pd.DataFrame(X_train).sample(frac = 0.1).values

In [44]:
#creating y_train_cat and y_test_cat\n",
X_train, X_test, y_train_cat, y_test_cat = model_selection.train_test_split(X, y_cat, train_size=0.70,test_size=0.30, random_state=101, shuffle = True)

In [45]:
y_train_sample = pd.DataFrame(y_train).sample(frac = 0.1).values

In [46]:
y_trainCat_sample = pd.DataFrame(y_train_cat).sample(frac = 0.1).values

In [47]:
X_test_sample = pd.DataFrame(X_test).sample(frac = 0.1).values

* Scaling

In [48]:
'''scaler = StandardScaler()
X = scaler.fit_transform(X)'''

'scaler = StandardScaler()\nX = scaler.fit_transform(X)'

In [49]:
'''y = (y - y.mean()) / y.std()'''

'y = (y - y.mean()) / y.std()'

In [50]:
from sklearn.metrics import r2_score

* Scaling PIPE

In [51]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [52]:
pipe = Pipeline([('scaler', StandardScaler())])
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler())])

Linear

In [53]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [54]:
linear_model = LinearRegression()

In [55]:
linear_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('linear_model', LinearRegression())])

In [56]:
linear_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_model', LinearRegression())])

In [57]:
y_pred_linear = linear_pipe.predict(X_test)

In [58]:
r2_score = linear_pipe.score(X_test, y_test)

In [59]:
r2_score

0.027835567454822185

* Pickling

In [60]:
with open('model_linear_pickle', 'wb') as linear_file:
    pickle.dump(linear_pipe, linear_file)

In [61]:
with open('model_linear_pickle', 'rb') as linear_file:
    model_linear = pickle.load(linear_file)

Linear_ElasticNet

In [62]:
from sklearn.linear_model import ElasticNet
elasnet_model = ElasticNet()

In [63]:
hyperparameters ={
    'elasnet_model__alpha': [1],
    'elasnet_model__l1_ratio': [0.0001]
}

In [64]:
elasnet_model_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('elasnet_model', ElasticNet())])
elasnet_model_grid = GridSearchCV(estimator=elasnet_model_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* Sample test to find Best Hyperparameters

In [65]:
'''elasnet_model_grid.fit(X_train_sample, y_trainCat_sample.ravel())
y_pred_randFor_class = elasnet_model_grid.predict(X_test_sample)'''

'elasnet_model_grid.fit(X_train_sample, y_trainCat_sample.ravel())\ny_pred_randFor_class = elasnet_model_grid.predict(X_test_sample)'

In [66]:
'''elasnet_model_grid.best_estimator_'''

'elasnet_model_grid.best_estimator_'

* Adjusted to Best Hyperparameters

In [67]:
elasnet_model_grid.fit(X_train, y_train_cat.ravel())

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('elasnet_model', ElasticNet())]),
             param_grid={'elasnet_model__alpha': [1],
                         'elasnet_model__l1_ratio': [0.0001]},
             scoring='r2')

In [68]:
y_pred_randFor_class = elasnet_model_grid.predict(X_test)

In [69]:
elasnet_model_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('elasnet_model', ElasticNet(alpha=1, l1_ratio=0.0001))])

In [70]:
r2_score = elasnet_model_grid.score(X_test, y_test_cat)
r2_score

0.021238705625799104

Polyfeats

In [71]:
from sklearn.preprocessing import PolynomialFeatures
poly_model = PolynomialFeatures()
poly_model_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('poly_model', PolynomialFeatures(2)), ('linear_reg', LinearRegression())])

* Sample test to find the Best Hyperparameters

In [72]:
poly_model_pipe.fit(X_train_sample, y_train_sample.ravel())
y_pred_poly = poly_model_pipe.predict(X_test_sample)

In [73]:
poly_model_pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('poly_model', PolynomialFeatures()),
                ('linear_reg', LinearRegression())])

In [74]:
r2_score = poly_model_pipe.score(X_test, y_test_cat)
r2_score

-2.5319028945327695e+27

Naives Bayes, GaussianNB Naive Bayes

In [75]:
from sklearn.naive_bayes import GaussianNB
NB_Gauss_model = GaussianNB()
NB_Gauss_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('NB_Gauss_model', GaussianNB())])
NB_Gauss_pipe.fit(X_train, y_train_cat)

Pipeline(steps=[('scaler', StandardScaler()), ('NB_Gauss_model', GaussianNB())])

In [76]:
y_pred_NBGauss = NB_Gauss_pipe.predict(X_test)

In [77]:
r2_score = NB_Gauss_pipe.score(X_test, y_test_cat)
r2_score

0.6008846546444369

* Pickling

In [78]:
with open('model_Gaussian_pickle', 'wb') as Gaussian_file:
    pickle.dump(NB_Gauss_pipe, Gaussian_file)

In [79]:
with open('model_Gaussian_pickle', 'rb') as Gaussian_file:
    model_Gaussian = pickle.load(Gaussian_file)

Random Forest Regressor

In [80]:
from sklearn.ensemble import RandomForestRegressor
random_forest_regression_model = RandomForestRegressor()

In [81]:
hyperparameters = {
    'rand_forrestReg__n_estimators': [1000],
    'rand_forrestReg__max_depth': [2],
    'rand_forrestReg__min_samples_split':[6], 
    'rand_forrestReg__bootstrap':[True],
    'rand_forrestReg__criterion' :['mse']
}

In [82]:
Forest_reg_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('rand_forrestReg', RandomForestRegressor())])
forest_reg_grid = GridSearchCV(estimator=Forest_reg_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* Sample test to find best hyperparameters

In [83]:
'''forest_reg_grid.fit(X_train_sample, y_train_sample.ravel())
y_pred_randFor_class = forest_reg_grid.predict(X_test_sample)'''

'forest_reg_grid.fit(X_train_sample, y_train_sample.ravel())\ny_pred_randFor_class = forest_reg_grid.predict(X_test_sample)'

In [84]:
#forest_reg_grid.best_estimator_

* Adjusted to Best Hyperparameters

In [85]:
forest_reg_grid.fit(X_train, y_train)
y_pred_randFo = forest_reg_grid.predict(X_test)

c:\Users\benny\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
c:\Users\benny\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
c:\Users\benny\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
c:\Users\benny\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
c:\Users\benny\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: C

In [86]:
forest_reg_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rand_forrestReg',
                 RandomForestRegressor(criterion='mse', max_depth=2,
                                       min_samples_split=6,
                                       n_estimators=1000))])

In [87]:
r2_score = forest_reg_grid.score(X_test, y_test)
r2_score

0.010367720729129726

* Pickling

In [88]:
with open('model_Forrest_reg_pickle', 'wb') as forest_reg_file:
    pickle.dump(forest_reg_grid, forest_reg_file)

In [89]:
with open('model_Forrest_reg_pickle', 'rb') as forest_reg_file:
    model_Forrest_reg_pickle = pickle.load(forest_reg_file)

SVM classification

In [90]:
from sklearn.svm import SVC, SVR, LinearSVC
model_svm_class = SVC()

In [91]:
hyperparameters = {'model_svm_class__kernel': ['linear'],
                   'model_svm_class__C':[1],
                   'model_svm_class__degree':[1]
}
SVC_grid_class = GridSearchCV(estimator=model_svm_class, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

In [92]:
svm_class_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('model_svm_class', SVC())])
smv_class_grid = GridSearchCV(estimator=svm_class_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* Sample test to find Best Hyperparameters

In [93]:
smv_class_grid.fit(X_train_sample, y_trainCat_sample.ravel())
y_pred_svm = smv_class_grid.predict(X_test_sample)
smv_class_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model_svm_class', SVC(C=1, degree=1, kernel='linear'))])

* Adjusted to Best Hyperparameters

In [94]:
smv_class_grid.fit(X_train,y_train_cat)
y_pred_svc_cat = smv_class_grid.predict(X_test)

In [95]:
smv_class_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model_svm_class', SVC(C=1, degree=1, kernel='linear'))])

In [96]:
r2_score = smv_class_grid.score(X_test, y_test_cat)
r2_score

-0.3995426030924909

* Pickling

In [97]:
with open('model_smv_class_pickle', 'wb') as smv_class_file:
    pickle.dump(smv_class_grid, smv_class_file)

In [98]:
with open('model_smv_class_pickle', 'rb') as smv_class_file:
    model_smv_class_pickle = pickle.load(smv_class_file)

XGBoost regressor

In [99]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [100]:
import xgboost as xgb

In [101]:
xg_reg = xgb.XGBRegressor()

In [102]:
hyperparameters = {
    'xg_reg__objective' : ['reg:squarederror'],
    'xg_reg__colsample_bytree':[0.1],
    'xg_reg__n_estimators': [3000],
    'xg_reg__max_depth': [4],
    'xg_reg__learning_rate': [0.0001],
    'xg_reg__alpha': [5]
}

In [103]:
svm_class_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('xg_reg', xgb.XGBRegressor())])
xgb_grid = GridSearchCV(estimator=svm_class_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* Sample Test to find Best Hyperparameters

In [104]:
xgb_grid.fit(X_train_sample, y_train_sample.ravel())
y_pred_xbg = xgb_grid.predict(X_test_sample)

In [105]:
xgb_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('xg_reg',
                 XGBRegressor(alpha=5, base_score=None, booster=None,
                              callbacks=None, colsample_bylevel=None,
                              colsample_bynode=None, colsample_bytree=0.1,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.0001, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=4, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=3000,
        

* Adjusted to Best Hyperparameters

In [106]:
xgb_grid.fit(X_train,y_train)
y_pred_xbg = xgb_grid.predict(X_test)

In [107]:
xgb_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('xg_reg',
                 XGBRegressor(alpha=5, base_score=None, booster=None,
                              callbacks=None, colsample_bylevel=None,
                              colsample_bynode=None, colsample_bytree=0.1,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.0001, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=4, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=3000,
        

In [108]:
r2_score = xgb_grid.score(X_test, y_test)
r2_score

-0.00041851692761607495

* Pickling

In [109]:
with open('model_xgb_grid_pickle', 'wb') as xgb_reg_file:
    pickle.dump(xgb_grid, xgb_reg_file)

In [110]:
with open('model_xgb_grid_pickle', 'rb') as xgb_reg_file:
    model_xgb_grid_pickle = pickle.load(xgb_reg_file)

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.